This workbook represent an end to end test of the Nillion network. It will test the following:
1. A user (patient) uploading their data in encrypted form and providing access to Monadic
2. Monadic giving compute access to Snipper, a third party
3. Snipper registering a program and running in on the patient's encrypted data

Before running this workbook, ensure Nillion Devnet is running. 

Install all Python prerequisites. 

In [1]:
%pip install nada-dsl==0.7.2
%pip install python-dotenv==1.0.0
%pip install nillion_client==0.1.1
%pip install cryptography==44.0.0

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Import all necessary libraries

In [3]:
import argparse
import asyncio
import os

from nillion_client import (
    InputPartyBinding,
    Network,
    NilChainPayer,
    NilChainPrivateKey,
    OutputPartyBinding,
    SecretInteger,
    UserId,
    Permissions,
    PermissionsDelta,
    VmClient,
    PrivateKey,
)
from dotenv import load_dotenv
from nillion_client.ids import UUID
from nillion_client.payer import DummyPayer
from nillion_client.permissions import (ComputePermissionCommand, ComputePermissions,ComputePermission)
import hashlib
from ecdsa import SigningKey, SECP256k1

Load the environment variables from a Devnet .env file

*Please make sure to replace the path with the correct path to your .env file*

In [4]:
import os

home_dir = os.path.expanduser("~")
env_path = os.path.join(home_dir, ".config", "nillion", "nillion-devnet.env")

print(f"Loading environment variables from {env_path}")

load_dotenv(env_path)

for key, value in os.environ.items():
    if key.startswith("NILLION_"):
        print(f"{key}: {value}")

Loading environment variables from /home/hello/.config/nillion/nillion-devnet.env
NILLION_CLUSTER_ID: 9e68173f-9c23-4acc-ba81-4f079b639964
NILLION_NILCHAIN_CHAIN_ID: nillion-chain-devnet
NILLION_NILCHAIN_JSON_RPC: http://127.0.0.1:48102
NILLION_NILCHAIN_REST_API: http://localhost:26650
NILLION_NILCHAIN_GRPC: http://localhost:26649
NILLION_GRPC_ENDPOINT: http://127.0.0.1:37939
NILLION_NILCHAIN_PRIVATE_KEY_0: 9a975f567428d054f2bf3092812e6c42f901ce07d9711bc77ee2cd81101f42c5
NILLION_NILCHAIN_PRIVATE_KEY_1: 1e491133b9408b39572a29f91644873decea554224b20e2b0b923aeb860a1c18
NILLION_NILCHAIN_PRIVATE_KEY_2: 980488572f235316cdb330191f8bafe4e635efbe88b3a40f5bee9bd21047c059
NILLION_NILCHAIN_PRIVATE_KEY_3: 612bb5173dc60d9e91404fcc0d1f1847fb4459a7d5160d63d84e91aacbf2ab2f
NILLION_NILCHAIN_PRIVATE_KEY_4: 04f5a984eeea9dce4e5e907da69c01a61568e3071b1a91cbed89225f9fd913b5
NILLION_NILCHAIN_PRIVATE_KEY_5: 5f992c58921f4af83b4c6b650c4914626664cd02020577b0ada49cfa00d2c8a4
NILLION_NILCHAIN_PRIVATE_KEY_6: 8f0297d

Read in some basic Nillion network information from the environment. 

In [8]:
use_testnet = False

if not use_testnet:
    cluster_id = os.getenv('NILLION_CLUSTER_ID')
    chain_id = os.getenv('NILLION_NILCHAIN_CHAIN_ID')
    nilvm_grpc_endpoint = os.getenv('NILLION_NILCHAIN_GRPC')
    payer_userkey_hex:str = os.getenv("NILLION_NILCHAIN_PRIVATE_KEY_0")
    monadic_seed = "monadic_seed"
    monadic_userkey_hex = hashlib.sha256(monadic_seed.encode()).digest().hex()
    snipper_seed = "snipper_seed"
    snipper_userkey_hex = hashlib.sha256(snipper_seed.encode()).digest().hex()
    patient_seed = "patient_seed"
    patient_userkey_hex = hashlib.sha256(patient_seed.encode()).digest().hex()
    network:Network = Network.from_config("devnet")
else:
    chain_id = "nillion-chain-testnet-1"
    chain_grpc_endpoint = "https://testnet-nillion-grpc.lavenderfive.com"
    nilvm_grpc_endpoint = "https://node-1.nucleus2.nillion-network.nilogy.xyz:14311"
    payer_userkey_hex = "11112217e1ba614ee927fdc38a31b686c7b56976aa92451e89f769b518416112"
    monadic_seed = "monadic_seed"
    monadic_userkey_hex = hashlib.sha256(monadic_seed.encode()).digest().hex()
    snipper_seed = "snipper_seed"
    snipper_userkey_hex = hashlib.sha256(snipper_seed.encode()).digest().hex()
    patient_seed = "patient_seed"
    patient_userkey_hex = hashlib.sha256(patient_seed.encode()).digest().hex()
    network:Network = Network(
      chain_grpc_endpoint=chain_grpc_endpoint,
      chain_id=chain_id,
      nilvm_grpc_endpoint=nilvm_grpc_endpoint  
    )
    
print(f"Chain ID: {chain_id}")
print(f"GRPC Endpoint: {nilvm_grpc_endpoint}")

network

Chain ID: nillion-chain-testnet-1
GRPC Endpoint: https://node-1.nucleus2.nillion-network.nilogy.xyz:14311


Network(chain_id='nillion-chain-testnet-1', chain_grpc_endpoint='https://testnet-nillion-grpc.lavenderfive.com', nilvm_grpc_endpoint='https://node-1.nucleus2.nillion-network.nilogy.xyz:14311')

Setup for the Monadic actor

In [9]:
monadic_userkey = NilChainPrivateKey(bytes.fromhex(monadic_userkey_hex))
monadic_payer = NilChainPayer(
    network,
    wallet_private_key=monadic_userkey,
    gas_limit=1000,
)
monadic_client = await VmClient.create(PrivateKey(bytes.fromhex(monadic_userkey_hex)), network, monadic_payer)
monadic_user_id = monadic_client.user_id

Setup for the Snipper actor

In [10]:
snipper_userkey = NilChainPrivateKey(bytes.fromhex(snipper_userkey_hex))
snipper_payer = NilChainPayer(
    network,
    wallet_private_key=snipper_userkey,
    gas_limit=1000,
)
snipper_client = await VmClient.create(PrivateKey(bytes.fromhex(snipper_userkey_hex)), network, snipper_payer)
snipper_user_id = snipper_client.user_id

Setup for the Patient actor

In [11]:
patient_userkey = NilChainPrivateKey(bytes.fromhex(patient_userkey_hex))
patient_payer = NilChainPayer(
    network,
    wallet_private_key=patient_userkey,
    gas_limit=1000,
)
patient_client = await VmClient.create(PrivateKey(bytes.fromhex(patient_userkey_hex)), network, patient_payer)
patient_user_id = patient_client.user_id

Payments set up for all network actions

In [12]:
payer_privKey = NilChainPrivateKey(bytes.fromhex(payer_userkey_hex))

payer = NilChainPayer(
    network,
    wallet_private_key=payer_privKey,
    gas_limit=10000000,
)
# We will identify ourselves with the pre-configured private key
payer_client= await VmClient.create(PrivateKey(bytes.fromhex(payer_userkey_hex)), network, payer)

Add funds to clients

In [13]:
funds_amount = 500000
bal = await payer_client.balance()
print(f"💰  Current Payer Balance: {bal.balance} uNIL")
if bal.balance < 400000:
   print(f"💰  Adding some funds to the executor client balance: {bal.balance} uNIL")
   await payer_client.add_funds(funds_amount)
bal = await patient_client.balance()
print(f"💰  Current Patient Balance: {bal.balance} uNIL")
if bal.balance < 400000:
   await payer_client.add_funds(500000, target_user=patient_user_id)
bal = await monadic_client.balance()
print(f"💰  Current monadic Balance: {bal.balance} uNIL")
if bal.balance < 400000:
   await payer_client.add_funds(500000, target_user=monadic_user_id)
bal = await snipper_client.balance()
print(f"💰  Current sniper Balance: {bal.balance} uNIL")
if bal.balance < 400000:
   await payer_client.add_funds(500000, target_user=snipper_user_id)


💰  Current Payer Balance: 500000 uNIL
💰  Current Patient Balance: 496598 uNIL
💰  Current monadic Balance: 500000 uNIL
💰  Current sniper Balance: 500000 uNIL


Configuration for the program to use. Change the contents of the below cell to use another program.

In [14]:
program_name = "double"
program_mir_path = f"binaries/double.nada.bin"
program = open(program_mir_path, "rb").read()

As a baseline, let's ensure that the user can store the program and run it on their own secret.

In [15]:
import os

os.environ['RUST_BACKTRACE'] = '1'

print("The patient is storing the program on the network")

program_id = await patient_client.store_program(
    program_name, program,
).invoke()

print(f"Program ID: {program_id}")

print("The patient is storing a secret on the network")

values = {"foo": SecretInteger(2) }

# Set permissions for the client to compute on the program
permissions = Permissions.defaults_for_user(patient_user_id).allow_compute(
            patient_user_id, program_id
        )

# Store a secret
store_id = await patient_client.store_values(
    values=values, ttl_days=5, permissions=permissions
).invoke()


print("The patient is running the program on the secret..")

party_name = "Party1"

input_bindings = [InputPartyBinding(party_name, patient_user_id)]
output_bindings = [OutputPartyBinding(party_name, [patient_user_id])]

computation_time_secrets = {}

# Compute on the secret
compute_id = await patient_client.compute(
    program_id,
    input_bindings,
    output_bindings,
    values=computation_time_secrets,
    value_ids=[store_id]
).invoke()

# 8. Return the computation result
print(f"The computation was sent to the network. compute_id: {compute_id}")
result = await patient_client.retrieve_compute_results(compute_id).invoke()
print(f"✅  Compute complete for compute_id {compute_id}")
print(f"🖥️  The result is {result}")
balance = await patient_client.balance()
print(f"💰  Final client balance: {balance.balance} uNIL")

The patient is storing the program on the network
Program ID: 4b367ad186fa672387ca7fb6ea9f75c13cc53a82/double/sha256/35f9b61f28b8e9aafbc987766f6ff549892aafe1d65256aa6feec84b1f5b6bf0
The patient is storing a secret on the network
The patient is running the program on the secret..
The computation was sent to the network. compute_id: e52ecb8b-26c5-4d49-9737-c12ec934ee1d
✅  Compute complete for compute_id e52ecb8b-26c5-4d49-9737-c12ec934ee1d
🖥️  The result is {'my_output': SecretInteger(4)}
💰  Final client balance: 496112 uNIL


Have Monadic store the program on the network

TODO: Also have Snipper save the program and replicate the computation

In [16]:
mondaic_program_id = await monadic_client.store_program(
    program_name, program,
).invoke()

print(f"Program ID: {mondaic_program_id}")


Program ID: ac9f9cea344cdbeaaa76ecca9eea25b21b0a1742/double/sha256/35f9b61f28b8e9aafbc987766f6ff549892aafe1d65256aa6feec84b1f5b6bf0


Have the Patient store a secret on the network such that:
- The Patient has all the regular persmissions
- Monadic is able to modify permissions and compute on the secret

TODO: Establish a chain by having Monadic actually assign compute access to Snipper

In [17]:
program_id = mondaic_program_id

values = {
        "foo": SecretInteger(2),
    }


# Set permissions for the client to compute on the program
permissions = Permissions.defaults_for_user(patient_user_id)\
    .allow_update(monadic_user_id)\
    .allow_compute(monadic_user_id, program_id)\


# Store a secret
new_store_id = await patient_client.store_values(
    values=values, ttl_days=5, permissions=permissions
).invoke()

Have Monadic run the stored program on the stored secret

TODO: Have Snipper run the stored program on the stored secret

In [18]:
import os

os.environ['RUST_BACKTRACE'] = 'full'

party_name = "Party1"

input_bindings = [InputPartyBinding(party_name, monadic_user_id)]
output_bindings = [OutputPartyBinding(party_name, [monadic_user_id])]

computation_time_secrets = {}

# Compute on the secret
compute_id = await monadic_client.compute(
    program_id,
    input_bindings,
    output_bindings,
    values=computation_time_secrets,
    value_ids=[new_store_id]
).invoke()

# 8. Return the computation result
print(f"The computation was sent to the network. compute_id: {compute_id}")
result = await monadic_client.retrieve_compute_results(compute_id).invoke()
print(f"✅  Compute complete for compute_id {compute_id}")
print(f"🖥️  The result is {result}")
balance = await monadic_client.balance()
print(f"💰  Final client balance: {balance.balance} uNIL")

The computation was sent to the network. compute_id: c5d48389-c300-41a2-99d1-202788f3ecb8
✅  Compute complete for compute_id c5d48389-c300-41a2-99d1-202788f3ecb8
🖥️  The result is {'my_output': SecretInteger(4)}
💰  Final client balance: 499995 uNIL


Update permissions on the value to give snipper bot compute perms

In [19]:
compute = ComputePermissions()
compute.permissions[snipper_user_id] = ComputePermission()
compute.permissions[snipper_user_id].program_ids.add(mondaic_program_id)
allowCompute = PermissionsDelta(compute=ComputePermissionCommand(grant=compute))
existingPermissions = await patient_client.retrieve_permissions(new_store_id).invoke()
await patient_client.update_permissions(new_store_id, allowCompute).invoke()
newPermissions = await patient_client.retrieve_permissions(new_store_id).invoke()

print("existing compute perms count", len(existingPermissions.compute.permissions))
print("new compute perms count", len(newPermissions.compute.permissions))


existing compute perms count 1
new compute perms count 2


Now have Snipper run the program

In [20]:
party_name = "Party1"

input_bindings = [InputPartyBinding(party_name, snipper_user_id)]
output_bindings = [OutputPartyBinding(party_name, [snipper_user_id])]

computation_time_secrets = {}

# Compute on the secret
compute_id = await snipper_client.compute(
    program_id,
    input_bindings,
    output_bindings,
    values=computation_time_secrets,
    value_ids=[new_store_id]
).invoke()

# 8. Return the computation result
print(f"The computation was sent to the network. compute_id: {compute_id}")
result = await snipper_client.retrieve_compute_results(compute_id).invoke()
print(f"✅  Compute complete for compute_id {compute_id}")
print(f"🖥️  The result is {result}")
balance = await snipper_client.balance()
print(f"💰  Final client balance: {balance.balance} uNIL")

The computation was sent to the network. compute_id: b93f96e7-45d0-4e74-92a0-cc4d547cc321
✅  Compute complete for compute_id b93f96e7-45d0-4e74-92a0-cc4d547cc321
🖥️  The result is {'my_output': SecretInteger(4)}
💰  Final client balance: 499996 uNIL
